# Imports

In [1]:
!pip install protobuf==3.20.3
!pip install transformers-interpret

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [2]:
import torch
from transformers_interpret import SequenceClassificationExplainer


import transformers
from datasets import load_from_disk

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pickle
import re

2025-11-25 21:05:05.454711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764104705.633154      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764104705.682698      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
torch.manual_seed(42)
np.random.seed(42)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

Running on device: cuda


# Model

In [5]:
model_path = '/kaggle/input/disorbert-finetuned-models/pytorch/default/1/finetuned-models/bert-base-cased-finetuned/checkpoint-13430'
tokenizer_bert = transformers.AutoTokenizer.from_pretrained(model_path, truncation=True, max_length=512)
model_bert = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, device_map='auto')
model_bert.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
tokenizer_bert

BertTokenizerFast(name_or_path='/kaggle/input/disorbert-finetuned-models/pytorch/default/1/finetuned-models/bert-base-cased-finetuned/checkpoint-13430', vocab_size=28996, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [7]:
pipe_bert = transformers.pipeline('text-classification', model=model_bert, tokenizer=tokenizer_bert,device_map='auto', max_length=512, truncation=True)
pipe_bert

Device set to use cuda:0


In [8]:
explainer = SequenceClassificationExplainer(model_bert, tokenizer_bert)
explainer.n_steps = 8
explainer.internal_batch_size = 1
explainer

In [9]:
text = "I feel stuck and useless. I hardly enjoy or look forward to things that life can offer. I don’t feel like my presence or anything I do is worthy. I feel like a burden to the people I love and care about. I just exist, I don’t even feel real.Sometimes I wish I could just pass in my sleep. I really don’t want to be here anymore."
print(pipe_bert(text))

word_attributions = explainer(text)
word_attributions.sort(key=lambda x : -x[1])
word_attributions[:10]

[{'label': 'LABEL_1', 'score': 0.9999932050704956}]


[('enjoy', 0.3059017062187195),
 ('stuck', 0.29398638010025024),
 ('useless', 0.27157309651374817),
 ('hardly', 0.15407729148864746),
 ('worthy', 0.13203264772891998),
 ('anymore', 0.0832769051194191),
 ('forward', 0.0829615518450737),
 ('i', 0.08032191544771194),
 ('burden', 0.07729493826627731),
 ('pass', 0.0745900571346283)]

In [10]:
text = "I recently turned 37 and I’ve accomplished absolutely nothing in life. No husband, kids pets, don’t own a home, f*ck all. Feeling really depressed about everything. I was severely bullied when I was younger and told I would never be successful, guess they were right. Was also told it was my own fault I was bullied whenever I tried to tell someone. The older I get the more I realize just how horribly treated I was. I’ve had nothing but horrible luck in my life. Last couple relationships I had were with severe alcoholics, and one of them gave me an incurable STD. Every guy I meet whenever I join a dating app who seems decent either doesn’t take me seriously, or turns out to be a raging alcoholic. On top of feeling lonely and depressed about everything, now my dad is really sick. He’s been in and out of the hospital and I’m afraid he doesn’t have much time left. There is absolutely nothing that could make me feel better at this point. I’m sick of people trying to tell me things will get better because they never do, or it’s not my fault because according to some people it is. I don’t want anyone to try and comfort me anymore as nothing ever gets better. I’m not comfortable talking to anyone mainly because when I tried to speak up when I was younger I was told everything was my fault.."
output = pipe_bert(text)
print(output)

word_attributions = explainer(text)
word_attributions.sort(key=lambda x : -x[1])

word_attributions[:10]

[{'label': 'LABEL_1', 'score': 0.9999886751174927}]


[('me', 0.2819424569606781),
 ('join', 0.24390453100204468),
 ('relationships', 0.23915717005729675),
 ('me', 0.21668580174446106),
 ('i', 0.19058889150619507),
 ('couple', 0.19004429876804352),
 ('raging', 0.18364350497722626),
 ('i', 0.1832517832517624),
 ('turned', 0.17629815638065338),
 ('meet', 0.16240063309669495)]

In [11]:
tokenizer_t5 = transformers.AutoTokenizer.from_pretrained("google-t5/t5-base")
model_t5 = transformers.AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base")
model_t5.eval()

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [12]:
import json

def createTemplateJSON(text, attributions, model_output):
    """
    text: str
    attributions: list of tuples  -> [('word', score), ...]
    model_output: list of dicts   -> [{'label': 'LABEL_1', 'score': 0.99}]
    """

    # Convert tuples like ('me', 0.28) into JSON-friendly lists ["me", 0.28]
    attribution_list = [[w, float(s)] for w, s in attributions]

    data = {
        "task": "Explain a psychological reasoning behind the model output.",
        "input_text": text,
        "model_output": model_output,
        "attribution_scores": attribution_list,
        "instruction": (
            "Provide a concise, evidence-based psychological interpretation explaining "
            "why the model may have predicted this output. Consider emotional tone, "
            "linguistic cues, cognitive patterns, or psychological markers present in the text."
        ),
        "response": ""
    }

    return json.dumps(data, indent=4)


In [13]:
class T5Pipeline:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, text, **gen_kwargs):
        inputs = self.tokenizer(text, return_tensors="pt")
        outputs = self.model.generate(**inputs, **gen_kwargs)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

pipe_t5 = T5Pipeline(model_t5, tokenizer_t5)
template = createTemplateJSON(text, word_attributions[:10], output)

print(pipe_t5(template, max_new_tokens=60))

 "task": "Explain a psychological reasoning behind the model output"  "response": ""  "response": ""  "response": ""  "respon


In [14]:
template

'{\n    "task": "Explain a psychological reasoning behind the model output.",\n    "input_text": "I recently turned 37 and I\\u2019ve accomplished absolutely nothing in life. No husband, kids pets, don\\u2019t own a home, f*ck all. Feeling really depressed about everything. I was severely bullied when I was younger and told I would never be successful, guess they were right. Was also told it was my own fault I was bullied whenever I tried to tell someone. The older I get the more I realize just how horribly treated I was. I\\u2019ve had nothing but horrible luck in my life. Last couple relationships I had were with severe alcoholics, and one of them gave me an incurable STD. Every guy I meet whenever I join a dating app who seems decent either doesn\\u2019t take me seriously, or turns out to be a raging alcoholic. On top of feeling lonely and depressed about everything, now my dad is really sick. He\\u2019s been in and out of the hospital and I\\u2019m afraid he doesn\\u2019t have much

# Dataset

In [15]:
from datasets import load_from_disk

test_dataset = load_from_disk("/kaggle/input/erisk25/erisk_processed/erisk_processed")['test']
test_dataset

Dataset({
    features: ['text', 'label', 'user'],
    num_rows: 3115
})

In [16]:
class TemplatePipeline:
    def __init__(self, pipe_bert, explainer, tokenizer, max_length=512):
        self.pipe_bert = pipe_bert
        self.explainer = explainer
        self.tokenizer = tokenizer
        self.max_length = max_length

    def truncate(self, text):
        """Safely truncate text to fit the BERT max token limit."""
        tokens = self.tokenizer.tokenize(text)

        # keep special tokens space and ensure CLS/SEP remain included
        if len(tokens) > (self.max_length - 2):  
            tokens = tokens[: (self.max_length - 2)]

        # reconstruct truncated text
        return self.tokenizer.convert_tokens_to_string(tokens)

    def get_bert_output(self, text):
        text = self.truncate(text)
        return self.pipe_bert(text)

    def get_attributions(self, text):
        text = self.truncate(text)
        word_attributions = self.explainer(text)
        word_attributions.sort(key=lambda x: -x[1])
        return word_attributions[:5]

    def get_template(self, text):
        text = self.truncate(text)
        bert_output = self.get_bert_output(text)
        attributions = self.get_attributions(text)
        return self.createTemplateJSON(text, attributions, bert_output)

    def createTemplateJSON(self, text, attributions, model_output):
        attribution_list = [[w, float(s)] for w, s in attributions]

        data = {
            "task": "Explain a psychological reasoning behind the model output.",
            "input_text": text,
            "model_output": model_output,
            "attribution_scores": attribution_list,
            "instruction": (
                "Provide a concise, evidence-based psychological interpretation explaining "
                "why the model may have predicted this output. Consider emotional tone, "
                "linguistic cues, cognitive patterns, or psychological markers present in the text."
            ),
            "response": ""
        }

        return json.dumps(data, indent=4)


In [17]:
test_df = test_dataset.to_pandas()

# Sample 50 positive and 50 negative
label_1_df = test_df[test_df["label"] == 1].sample(n=300, random_state=42)
label_0_df = test_df[test_df["label"] == 0].sample(n=300, random_state=42)

# Combine
balanced_test_subset = pd.concat([label_1_df, label_0_df]).reset_index(drop=True)

balanced_test_subset = (
    balanced_test_subset.sample(frac=1, random_state=42).reset_index(drop=True)
)

balanced_test_subset.head(), balanced_test_subset.shape

(                                                text  label             user
 0  eally bored in my current dead-end job and it'...      1  subject_pgmMwDj
 1   This is AI generated This is AI generated Thi...      0  subject_B5WhFd0
 2  ectrical Engineering over prospects. Any tips ...      0  subject_hxc2Sft
 3  st need more time to get over it, but knowing ...      1  subject_w4ToS2n
 4  t some mundane stuff. Then he said he was goin...      1  subject_BFe7jfH,
 (600, 3))

In [18]:
# template_pipeline = TemplatePipeline(pipe_bert, explainer)
template_pipeline = TemplatePipeline(pipe_bert, explainer, tokenizer_bert)
balanced_test_subset['prompt'] = balanced_test_subset['text'].apply(template_pipeline.get_template)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [19]:
import json

def save_prompts_from_dataframe(df, column_name="prompt", filename="prompts.json"):
    """Save only a specific column from a dataframe into a JSON list."""
    
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in dataframe.")
    
    # Extract prompts as list
    prompt_list = df[column_name].tolist()
    
    # Save to JSON file
    with open(filename, "w") as f:
        json.dump(prompt_list, f, indent=4)

    print(f"Saved {len(prompt_list)} prompts to {filename}")


In [20]:
save_prompts_from_dataframe(balanced_test_subset, "prompt", "bert_generated_prompts.json")

Saved 600 prompts to bert_generated_prompts.json
